In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
books_df = pd.read_csv('BX-Books.csv',encoding='latin-1', delimiter=',',error_bad_lines = False)
ratings_df = pd.read_csv('BX-Book-Ratings.csv',encoding='latin-1', delimiter=',',error_bad_lines = False)
print(books_df.info())

C:\Users\edwar\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3378: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 10824: expected 8 fields, saw 9\nSkipping line 14769: expected 8 fields, saw 9\nSkipping line 14936: expected 8 fields, saw 9\nSkipping line 15690: expected 8 fields, saw 10\nSkipping line 17320: expected 8 fields, saw 9\nSkipping line 18095: expected 8 fields, saw 9\nSkipping line 28443: expected 8 fields, saw 9\nSkipping line 33098: expected 8 fields, saw 9\nSkipping line 36394: expected 8 fields, saw 12\nSkipping line 42628: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\nSkipping line 52174: expected 8 fields, saw 9\nSkipping line 56901: expected 8 fields, saw 10\nSkipping line 58600: expected 8

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271264 entries, 0 to 271263
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271264 non-null  object
 1   Book-Title           271264 non-null  object
 2   Book-Author          271263 non-null  object
 3   Year-Of-Publication  271264 non-null  object
 4   Publisher            271262 non-null  object
 5   Image-URL-S          271264 non-null  object
 6   Image-URL-M          271264 non-null  object
 7   Image-URL-L          271261 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
None


In [3]:
userInput = [{'Title':'Classical Mythology', 'Rating':'0'},
            {'Title':'Clara Callan', 'Rating':'5'},
            {'Title':'Jane Doe','Rating':'5'},
            {'Title':'The Testament','Rating':'0'},
            {'Title':'Decorative Stamping: Hundreds of Projects for Your Home','Rating':'7'}]
        
inputBooks = pd.DataFrame(userInput)
print(inputBooks)

                                               Title Rating
0                                Classical Mythology      0
1                                       Clara Callan      5
2                                           Jane Doe      5
3                                      The Testament      0
4  Decorative Stamping: Hundreds of Projects for ...      7


In [4]:
inputID = books_df[books_df['Book-Title'].isin(inputBooks['Title'].tolist())]
inputBooks = pd.merge(inputID,inputBooks,left_index=True, right_index=True)
inputBooks = inputBooks[['ISBN','Title','Rating']]
print(inputBooks)

         ISBN                Title Rating
0  0195153448  Classical Mythology      0
1  0002005018         Clara Callan      5


In [5]:
userSubset = ratings_df[ratings_df['ISBN'].isin(inputBooks['ISBN'].tolist())]
print(userSubset.groupby('ISBN').count())

            User-ID  Book-Rating
ISBN                            
0002005018       14           14
0195153448        1            1


In [6]:
userSubsetGroup = userSubset.groupby(['ISBN'])

def take_3_elem(x):
    # print (len(x[1]))
    return len(x[1])
    
userSubsetGroup = sorted(userSubsetGroup, key=take_3_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:3])

[('0002005018',          User-ID        ISBN  Book-Rating
9563           8  0002005018            5
43178      11400  0002005018            0
45340      11676  0002005018            8
188244     41385  0002005018            0
283152     67544  0002005018            8
354414     85526  0002005018            0
399690     96054  0002005018            0
490342    116866  0002005018            9
508051    123629  0002005018            9
734221    177458  0002005018            0
828273    200273  0002005018            8
871535    210926  0002005018            9
903613    219008  0002005018            7
1096853   263325  0002005018            6), ('0195153448',       User-ID        ISBN  Book-Rating
9561        2  0195153448            0)]


In [7]:
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:

    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='ISBN')
    inputBooks = inputBooks.sort_values(by='ISBN')

    #Get the N for the formula
    nRatings = len(group)

    #Get the review scores for the movies that they both have in common
    temp_df = inputBooks[inputBooks['ISBN'].isin(group['ISBN'].tolist())]

    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['ISBN'].tolist()

    tempR = []
    for i in tempRatingList:
        tempR.append(int(i))

    tempG = []
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['ISBN'].tolist()

    for i in tempGroupList:
        tempG.append(int(i))

    
    #Now let's calculate the pearson correlation between two users, so called, x and y manually (check the formula from week 7 slide)
    Sxx = sum([pow(i,2) for i in tempR]) - pow(sum(tempR),2)/int(nRatings)
    Syy = sum([pow(i,2) for i in tempG]) - pow(sum(tempG),2)/int(nRatings)
    Sxy = sum( i*j for i, j in zip(tempR, tempG)) - sum(tempR)*sum(tempG)/int(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [8]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex      userId
0                0  0002005018
1                0  0195153448


In [9]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

   similarityIndex      userId
0                0  0002005018
1                0  0195153448


In [10]:
topUsersRating=topUsers.merge(ratings_df, left_index=True, right_index=True)
print(topUsersRating.head(100))

   similarityIndex      userId  User-ID        ISBN  Book-Rating
0                0  0002005018   276725  034545104X            0
1                0  0195153448   276726  0155061224            5


In [11]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Book-Rating']
print(topUsersRating.head())

   similarityIndex      userId  User-ID        ISBN  Book-Rating  \
0                0  0002005018   276725  034545104X            0   
1                0  0195153448   276726  0155061224            5   

   weightedRating  
0               0  
1               0  


In [12]:
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

            sum_similarityIndex  sum_weightedRating
ISBN                                               
0155061224                    0                   0
034545104X                    0                   0


In [13]:
recommendation_df = pd.DataFrame()

recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index
print(recommendation_df.head(10))

            weighted average recommendation score        ISBN
ISBN                                                         
0155061224                                    NaN  0155061224
034545104X                                    NaN  034545104X


In [14]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df)

            weighted average recommendation score        ISBN
ISBN                                                         
0155061224                                    NaN  0155061224
034545104X                                    NaN  034545104X


In [15]:
recommended_books=books_df.loc[books_df['ISBN'].isin(recommendation_df['ISBN'])]

#we don't want to recommend the same movie
recommended_books=recommended_books.loc[~recommended_books.ISBN.isin(userSubset['ISBN'])]

recommended_books = recommended_books.drop(['Year-Of-Publication','Publisher','Image-URL-S','Image-URL-M','Image-URL-L'], axis=1)

print(recommended_books)

              ISBN            Book-Title Book-Author
2966    034545104X  Flesh Tones: A Novel  M. J. Rose
225735  0155061224      Rites of Passage  Judith Rae
